Handle Raw Data
###

- *Expects*: PICKLE
- *Returns*: CSV

This file takes in the raw data, which has been downloaded from the server, and passed into the digital twin. It can then be downloaded from the download link. This is done for 2 reasons:
- To reduce processing memory requires on the server
- To use SQL to filter by date, and thus reduce memory load on analysis machines

Running the cells in this notebook will do the following:
- Clean the data and structure it into a per-participant format
- De-duplicate the data, according to pre-specified criteria [TODO]
- Parse the data into round-level data

In [1]:
import json
import pandas as pd
import urllib.parse
from collections import Counter
import numpy as np
import pickle

In [2]:
### PARAMETERS
# ------------

# Pickle path
picklePath = "1706270670.pickle"
# The path to where the csv should be saved
csvPath = "rw_24_all.csv"
# Path to the duplicate file
duplicatesPath = "duplicates.csv"

# ------------

In [3]:
# Open as pickle
with open(picklePath, 'rb') as handle:
    qs = pickle.load(handle)

jsonData = qs.decode('utf8').replace("'", '"')
data = json.loads(jsonData)

In [5]:
def clean(rawData):
    cleanData = {}
    for user in rawData:
        # pk = user['id']
        # user = userAll['fields']
        tmp = {
            'pk' : user['id'],
            'userId' : urllib.parse.quote(user['userId']),
            'rawData' : json.loads(user['rawData']),
            'sdata' : None,
            # 'edata' : user['edata'],
            # 'parameters' : user['parameters'],
            'totalAttempts' : None,
            'completed' : user['completeAttempt'],
            # 'lastCompletedRound' : None,
            # 'lastTrialGame' : None,
            # 'finalRooms' : [],
            # 'urlParameters' : user['urlParameters'],
            'timestamps' : [],
            'timeCreated' : user['timeCreated'],
            'lastModified' : user['lastModified']
        }
        # Check how many attempts the user has had
        tmp['totalAttempts'] = len(tmp['rawData'])
        # Store timestamp/s
        tmp['timestamps'] = list(tmp['rawData'].keys())
        # If it's one, check if it's complete
        if tmp['totalAttempts'] == 1:
            # Get the attempt timestamp
            timestamp = list(tmp['rawData'].keys())[0]
        elif tmp['totalAttempts'] > 1:
            # For multiple attempts, find the occurence with the highest number of completed trial_layouts
            indAttempts = [] # individual attempts
            for i in range(tmp['totalAttempts']):
                # Get the sdata for this timestamp, and get the length of the expt_index array
                try:
                    attNum = len(json.loads(tmp['rawData'][tmp['timestamps'][i]]['sdata'])['expt_index'])
                    indAttempts.append(attNum)
                except:
                    indAttempts.append(0)
            # Get the index of the value with the greatest magnitude
            timestamp = tmp['timestamps'][np.argmax(indAttempts)]
            
        # Use the timestamp to add sdata to tmp
        try:
            tmp['sdata'] = json.loads(tmp['rawData'][timestamp]['sdata'])
        except:
            if tmp['rawData'][timestamp]['sdata'] == None or len(tmp['rawData'][timestamp]['sdata']) == 0:
                tmp['sdata'] = None
        if tmp['sdata'] != None:
            # Check if complete by:
            #    - trial_layout == 92 or
            #    - trial_game == 80
            # if len(Counter(tmp['sdata']['trial_layout']).keys()) >= 92:
            if max(np.array(tmp['sdata']['trial_game'], dtype=np.float64)) >= 80:
                tmp['completed'] = True
                tmp['lastCompletedRound'] = len(tmp['sdata']['trial_game'])
            else:
                tmp['completed'] = False
                tmp['lastCompletedRound'] = len(Counter(tmp['sdata']['trial_layout']).keys())
            # Store how many trial_games they've seen
            tmp['lastTrialGame'] = int(tmp['sdata']['trial_game'][-1])
        else:
            tmp['completed'] = False
            tmp['lastCompletedRound'] = 0
            
        cleanData[tmp['pk']] = tmp
    print("Dataset ready.")
    return cleanData

In [6]:
def getRounds(cleanData):
    rounds = {}
    count = 0
    for user in cleanData:
        u = cleanData[user]
        us = u['sdata']
        if us is not None:
            for i in range(len(u['sdata']['expt_index'])):
                rounds[str(count)] = {
                    "pk" : u['pk'],
                    "id" : u['userId'],
                    "iv" : None if (u['urlParameters'] is None or u['urlParameters']['iv'] == None) else urllib.parse.quote(u['urlParameters']['iv']),
                    "tag" : None if (u['urlParameters'] is None or u['urlParameters']['tag'] == None) else urllib.parse.quote(u['urlParameters']['tag']),
                    "expt_index": us['expt_index'][i],
                    "expt_trial": us['expt_trial'][i],
                    "trial_layout" : us['trial_layout'][i],
                    "trial_level" : us['trial_level'][i],
                    "trial_solved" : us['trial_solved'][i],
                    "trial_attempts" : us['trial_attempts'][i],
                    "trial_game" : us['trial_game'][i],
                    "trial_transfer" : us['trial_transfer'][i],
                    "trial_test" : us['trial_test'][i],
                    "round_start_time" : None,
                    "round_end_time" : None,
                    "last_room" : None,
                    "roundAttempted" : None,
                    "gameComplete" : u["completed"],
                }

                # Insert start time, end time, and roundAttempted
                if len(us["resp"][str(i)]["timestamp"]) != 0:
                    # If an attempt has been made
                    rounds[str(count)]["round_start_time"] = us["resp"][str(i)]["timestamp"][0] - us["resp"][str(i)]["reactiontime"][0]
                    rounds[str(count)]["round_end_time"] = us["resp"][str(i)]["timestamp"][-1]
                    rounds[str(count)]["roundAttempted"] = True
                else:
                    rounds[str(count)]["roundAttempted"] = False
                    
                # Compute final room position as [x, y]. Scale is 0->10 and includes 2 walls
                if len(us["resp"][str(i)]["xloc"]) != 0 and len(us["resp"][str(i)]["yloc"]) != 0:
                    xloc = us["resp"][str(i)]["xloc"][-1]
                    yloc = us["resp"][str(i)]["yloc"][-1]
                    rounds[str(count)]["last_room"] = []
                    if xloc < 3:
                        rounds[str(count)]["last_room"].append(0)
                    elif 3 < xloc < 7:
                        rounds[str(count)]["last_room"].append(1)
                    elif 7 < xloc < 11:
                        rounds[str(count)]["last_room"].append(2)
                    if yloc < 3:
                        rounds[str(count)]["last_room"].append(0)
                    elif 3 < yloc < 7:
                        rounds[str(count)]["last_room"].append(1)
                    elif 7 < yloc < 11:
                        rounds[str(count)]["last_room"].append(2)
                    rounds[str(count)]["last_room"] = str(rounds[str(count)]["last_room"]).replace(",", "-")

                # Increase round count
                count += 1

    # Store as dataframe
    rounds = pd.DataFrame.from_dict(rounds, orient="index")

    return rounds


In [7]:
cleanData = clean(data)

Dataset ready.


In [ ]:
for i in cleanData:
    print(cleanData[i].keys())
    break

dict_keys(['pk', 'userId', 'rawData', 'sdata', 'edata', 'parameters', 'totalAttempts', 'completed', 'lastCompletedRound', 'lastTrialGame', 'finalRooms', 'urlParameters', 'timestamps', 'timeCreated', 'lastModified'])


In [309]:
cleanData_df = pd.DataFrame.from_dict(cleanData, orient="index")

In [311]:
cleanData_df.to_csv("../data/rise_year4_clean.csv", index=False)

In [396]:
def processDuplicates(cleanData, format):
    """
    The most recent version of the deduplication processor
    """
    duplicates = pd.read_csv(duplicatesPath)
    format = format.lower()
    assert format in ["None", None, "research", "rise", "all"], f"Unrecognised format for deduplication: {format} - allowed types are None, RISE, research, all. See docs for more information."
    # convert the cleanData from JSON to dataframe
    clean_df = pd.DataFrame.from_dict(cleanData, orient="index")
    # Make a list of riseId : [userIds] that contains single-players, and duplicate players
    counts = Counter(duplicates["USER_ID"])
    # map riseId to [userIds]
    ids = {id : list(duplicates.loc[duplicates["USER_ID"] == id]["ID"]) for id in counts}
    # Get only duplicate users
    duplicateHashes = {i : ids[i] for i in ids if len(ids[i]) >= 2}
    print(f"# Unique players: {len(ids)} -- # of those who repeated: {len(duplicateHashes)}")

    riseAttempts = [] # the first complete if available - if not, the first incomplete
    researchAttempts = []
    allAttempts = [] # for debugging, not important
    # take the first complete attempt - if none found, take just the first incomplete attempt
    # for each attempt made by one individual, check if they completed
    mapping = []
    i = 0
    for attempt in ids:
        # Get a df for just this participant
        repeats = clean_df.loc[clean_df["userId"].isin(ids[attempt])]
        completes = repeats.loc[repeats["completed"] == True]
        if (len(repeats) == 0):
            continue
        if len(completes) == 0:
            # no completed attempts, take just the first attempt
            firstTimestamp = min(list(repeats["timeCreated"]))
            # Returns the hashed ID associated with the attempt
            riseAttempts.append(list(repeats.loc[repeats["timeCreated"] == firstTimestamp]["userId"])[0])
            # Store a lookup of this hashed ID and the unique attempt number it was
            mapping.append({"userId" : riseAttempts[-1], "uniqueAttemptNumber" : 0})
        elif len(completes) == 1:
            # if 1 complete attempt - NB: this could be in any position, not necessarily first attempted
            riseAttempts.append(list(completes["userId"])[0])
            timestamp = list(completes["timeCreated"])[0]
            # This complete may not be the first attempt, so let's see which attempt number it is
            # See which attempt number this is:
            #   - Sort the timestamps in order, and get the index of our timestamp in that list
            uniqueAttemptNumber = list(repeats["timeCreated"].sort_values()).index(timestamp)
            # Store a lookup of this hashed ID and the unique attempt number it was
            mapping.append({"userId" : riseAttempts[-1], "uniqueAttemptNumber" : uniqueAttemptNumber})
        else:
            # if multiple complete attempts found - take the first for rise
            firstTimestamp = min(list(completes["timeCreated"]))
            riseAttempts.append(list(completes.loc[completes["timeCreated"] == firstTimestamp]["userId"])[0])
            # Store a lookup of this hashed ID and the unique attempt number it was
            mapping.append({"userId" : riseAttempts[-1], "uniqueAttemptNumber" : 0})

        # if more than one attempt, save which attempt number in total this is
        if len(repeats) > 1:
            # order the creation timestamps
            repeats["timeCreated"].sort_values()
            pass

        # For research purposes, we want to take only the very first attempt
        firstTimestamp = min(list(repeats["timeCreated"]))
        tmp = repeats.loc[repeats["timeCreated"] == firstTimestamp]["userId"]

        researchAttempts.append(list(tmp)[0])
        allAttempts.append(ids[attempt])

        # i += 1
        # if i == 10:
        #     break

    if format == "rise":
        # merge the id : unique attempt number mapping
        mapping_df = pd.DataFrame.from_dict(mapping)
        # keep only the rise ones in cleanData
        return clean_df.loc[clean_df["userId"].isin(list(riseAttempts))].merge(mapping_df)
            
    elif format == "research":
        # keep only the research ones in cleanData
        return clean_df.loc[clean_df["userId"].isin(researchAttempts)]

In [397]:
dedupped_df = processDuplicates(cleanData, "research")

# Unique players: 10126 -- # of those who repeated: 1763


In [398]:
dedupped_df.to_csv("../data/rise_year4_clean_researchDeduplicated.csv")

In [319]:
Counter(list(dedupped_df["completed"]))

Counter({True: 6568, False: 3556})

In [320]:
cleanDataDedupped = pd.DataFrame.to_dict(dedupped_df, orient="index")
rounds = getRounds(cleanDataDedupped)

In [321]:
rounds.to_csv("../data/rise_year4_riseDeduplicated_rounds.csv")

In [6]:
# Clean the raw data
cleanData = clean(data)
# Load in duplicate crosswalk
duplicates = pd.read_csv("y2/duplicates.csv")
# Filter duplicates
filteredCleanData = processDuplicates(cleanData, duplicates)
# Convert filtered data from DF to dict
cleanData = pd.DataFrame.to_dict(filteredCleanData, orient="index")
# Get round-level data
rounds = getRounds(cleanData)

Dataset ready.


In [25]:
rounds.to_csv(csvPath, index=False)

In [17]:
Counter(filteredCleanData["completed"])

Counter({False: 6196, True: 6639})